In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

In [ ]:


widgets.Box(
    [
        widgets.Label(value='Pizza topping with a very long label:'),
        widgets.RadioButtons(
            options=[
                'pepperoni',
                'pineapple',
                'anchovies',
                'and the long name that will fit fine and the long name that will fit fine and the long name that will fit fine '
            ],
            layout={'width': 'max-content', 'orientation':'horizontal'},
        )
    ]
)



widgets.Select(
    options=['Linux', 'Windows', 'OSX'],
    value='OSX',
    # rows=10,
    description='OS:',
    disabled=False
)



widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)



In [ ]:
global x
x=1.2

In [ ]:
text = widgets.Text()
display(text)

def handle_submit(sender):
    print(text.value)
    
text.on_submit(handle_submit)

In [ ]:
def f(y):
    global x
    x = y
    print(y)
interact(f, y=10.0)

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import h5py, tifffile
import numpy as np
import skimage.morphology as skm
import ipywidgets.widgets as widgets
from scipy.ndimage import gaussian_filter
from io import BytesIO

In [ ]:
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/3D_trial_reg_scan_id_55936-55956_2020-03-22-22-34-58.h5'
f = h5py.File(fn, 'r')
#global img
img = f['/processed_XANES3D/proc_spectrum/spectrum_whiteline'][300, ...]
roi = f['/registration_results/reg_parameters/slice_roi'][:]
f.close()

#global img_sli
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/recon_fly_scan_id_55946/recon_fly_scan_id_55946_{}.tiff'.format(str(roi[4]+300).zfill(5))
img_sli = tifffile.imread(fn)[roi[0]:roi[1], roi[2]:roi[3]]
#fig, ax = plt.subplots(111)
#ax[0].imshow(img)
#image_widget = widgets.Image(image=img)
def func(idx=300, thres1=8.345, thres2=1e-4, gaussian_w=5, blur=10):
    fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/3D_trial_reg_scan_id_55936-55956_2020-03-22-22-34-58.h5'
    f = h5py.File(fn, 'r')
    img = f['/processed_XANES3D/proc_spectrum/spectrum_whiteline'][idx, ...]
    roi = f['/registration_results/reg_parameters/slice_roi'][:]
    f.close()
    fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/recon_fly_scan_id_55946/recon_fly_scan_id_55946_{}.tiff'.format(str(roi[4]+idx).zfill(5))
    img_sli = tifffile.imread(fn)[roi[0]:roi[1], roi[2]:roi[3]]
    print('thres1:', thres1, 'thres2:', thres2)
    
    #plt.imshow(img*(img<thres1)*(img_sli>thres2))
    plt.imshow(img*(img<thres1)*skm.binary_dilation((gaussian_filter(img_sli, gaussian_w)>thres2), selem=skm.disk(blur)), vmin=8.345, vmax=8.355)
    plt.show()
    #image_widget.image= img
widgets.interact(func, idx=(0, 645), thres1=(8.33, 8.37, 5e-4), thres2=(-2e-3, 4e-3, 1e-5), blur=(0, 30))




In [ ]:
fn1 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/3D_trial_reg_scan_id_55936-55956_2020-03-22-22-34-58.h5'
f = h5py.File(fn1, 'r')
img = np.float32(f['/processed_XANES3D/proc_spectrum/spectrum_whiteline'])[300, ...]
roi = f['/registration_results/reg_parameters/slice_roi'][:]
fn2 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/recon_fly_scan_id_55946/recon_fly_scan_id_55946_{}.tiff'.format(str(roi[4]+300).zfill(5))
img_sli = np.float32(tifffile.imread(fn2)[roi[0]:roi[1], roi[2]:roi[3]])
for ii in range(roi[4], roi[5]):
#for ii in range(roi[4]+200, roi[4]+210):
    img[:] = f['/processed_XANES3D/proc_spectrum/spectrum_whiteline'][ii-roi[4], ...]
    fn2 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/recon_fly_scan_id_55946/recon_fly_scan_id_55946_{}.tiff'.format(str(ii).zfill(5))
    img_sli[:] = np.float32(tifffile.imread(fn2))[roi[0]:roi[1], roi[2]:roi[3]] 
    img[:] = np.float32(img*(img<8.353)*skm.binary_dilation((gaussian_filter(img_sli, 15)>1e-3), selem=skm.disk(0)))[:]
    #img[:] = np.float32(img*(img<8.353)*skm.binary_dilation((gaussian_filter(img_sli, 15)>1e-3), selem=skm.disk(15)))[:]
    fn3 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_slice_{}.tiff'.format(str(ii).zfill(5))
    tifffile.imsave(fn3, img)
    print(ii)
f.close()

In [ ]:
roi

In [ ]:
import glob, tifffile
from scipy.ndimage import zoom
import numpy as np
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_slice_{}.tiff'
img_ids = sorted(glob.glob(fn.format('*')))
img = np.float32(tifffile.imread(fn.format('600'.zfill(5))))
vol0 = np.ndarray([len(img_ids), img.shape[0], img.shape[1]], dtype=np.float32)
cnt = 0
for ii in img_ids:
    img[:] = np.float32(tifffile.imread(ii))[:]
    vol0[cnt, :] = img[:]
    cnt += 1
    print(ii)
vol1 = np.float32(zoom(vol0, (0.5, 0.5, 0.5)))
fn1 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5.tif'
tifffile.imsave(fn1, vol1)

In [ ]:
fn1 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5.tif'
vol1 = tifffile.imread(fn1)
fn3 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_3.tif'   
vol3 = tifffile.imread(fn3)
vol0 = np.ndarray(vol1.shape, dtype=np.float32)

for ii in range(vol1.shape[0]):
    vol0[ii, ...] = (vol1[ii]*(vol3[ii]>3))[:]
vol0[:] = np.where(vol0<8.343, 8, vol0)[:]
vol0[:] = np.where(vol0>8.353, 8.353, vol0)[:]
fn4 = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_4.tif'     
tifffile.imsave(fn4, vol0)

In [ ]:
print(vol0.max(), vol0.min())

In [ ]:
import scipy.ndimage as sn
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_4.tif'
img = tifffile.imread(fn)
sli = img[0]
def func(idx=300, thres1=8.345, median=3, close_w=5):
    print('thres1:', thres1)
    if (median != 0) & (close_w != 0):
        sli[:] = sn.median_filter(img[idx], median)[:]
        plt.imshow(skm.binary_closing((sli<thres1)&(sli>8.1), selem=skm.disk(close_w)).astype(np.int8))
    elif (median == 0) & (close_w != 0):
        plt.imshow(skm.binary_closing(((img[idx])<thres1)&(img[idx])>8.1), selem=skm.disk(close_w)).astype(np.int8)
    elif (median != 0) & (close_w == 0):
        sli[:] = sn.median_filter(img[idx], median)[:]
        plt.imshow(((sli<thres1)&(sli>8.1)).astype(np.int8))
    else:
        plt.imshow(((img[idx]<thres1)&(img[idx]>8.1)).astype(np.int8))
    plt.show()
widgets.interact(func, idx=(0, 322), thres1=(8.31, 8.37, 5e-4), median=(0, 10, 1), close_w=(0, 30, 1))



In [ ]:
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_4.tif'
img = tifffile.imread(fn)
median = 10
close_w = 3
thres1 = 8.349
sli = img[0]
mask = np.ndarray([img.shape[1], img.shape[2]])
for ii in range(img.shape[0]):
    sli[:] = sn.median_filter(img[ii], median)[:]
    mask[:] = skm.binary_closing((sli<thres1)&(sli>8.1), selem=skm.disk(close_w)).astype(np.int8)[:]
    img[ii] = (sn.median_filter(mask*img[ii], median) + sn.median_filter((1-mask)*img[ii], median))[:]
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_4_median_filter_10_close_3_thresholde_8.349.tif'
tifffile.imsave(fn, img)

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
img1 = (img*(img<8.3525)*skm.binary_dilation((img_sli>2e-3), selem=skm.disk(15)))
plt.subplot(131)
plt.imshow(img)
plt.subplot(132)
plt.imshow(img_sli)
plt.subplot(133)
plt.imshow(img1)

In [ ]:
plt.subplot(121)
plt.imshow(vol0[300,...])
plt.subplot(122)
plt.imshow(vol1[200,...])

In [ ]:
import os
#dir(os.path)
fn = '/NSLS2/xf18id1/users/2020Q1/TONGCHAO_LIU_Proposal_000000/masked_analyzed_3D_spectrum/xanes3D_downsampled_0.5_4_median_filter_10_close_3_thresholde_8.349.tif'
print(os.path.splitext(fn))

In [ ]:
import h5py
#dir(h5py)
help(h5py.VirtualLayout)

In [ ]:
%matplotlib qt
import tomopy
import h5py
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
fn = "/NSLS2/xf18id1/users/2020Q1/ENYUAN_Proposal_305409/fly_scan_id_29671.h5"
f = h5py.File(fn, 'r')
img = f['/img_tomo'][:, 500:501, :]
f.close()

In [ ]:
plt.imshow(img[:,0,:])

In [ ]:
plt.plot(img[:, 0, :].mean(axis=1))

In [ ]:
f = []
len(f)

In [3]:
rec_sli_start = 0
rec_sli_end = 1800
rec_chunk_sz = 200
rec_margin = 15
dim = [1000, 2160]

if rec_sli_start is None:
    rec_sli_start = 0
if rec_sli_end is None:
    rec_sli_end = dim[1]

if rec_chunk_sz >= (rec_sli_end-rec_sli_start):
    rec_chunk_sz = (rec_sli_end-rec_sli_start)
    num_chunk = 1
else:
    num_chunk = int((rec_sli_end-rec_sli_start)/(rec_chunk_sz-2*rec_margin)) + 1

print(num_chunk)

for ii in range(num_chunk):
    if ii == 0:
        sli_start = rec_sli_start
        sli_end = rec_sli_start + rec_chunk_sz
    else:
        sli_start = rec_sli_start + ii*(rec_chunk_sz-2*rec_margin)
        sli_end = sli_start + rec_chunk_sz
        if sli_end > rec_sli_end:
            sli_end = rec_sli_end
        if sli_end > dim[1]:
            sli_end = dim[1]

    if (sli_end - sli_start) <= rec_margin:
        break
        
    print(ii, sli_start, sli_end)

11
0 0 200
1 170 370
2 340 540
3 510 710
4 680 880
5 850 1050
6 1020 1220
7 1190 1390
8 1360 1560
9 1530 1730
10 1700 1800


In [ ]:
from ipywidgets import widgets


widgets.SelectMultiple(
    options=['Apples', 'Oranges', 'Pears'],
    value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)

In [ ]:
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')





grid = GridspecLayout(4, 3, height='300px')
grid[:3, 1:] = create_expanded_button('One', 'info')
grid[:, 0] = create_expanded_button('Two', 'info')
grid[3, 1] = create_expanded_button('Three', 'info')
grid[3, 2] = create_expanded_button('Four', 'info')

grid









In [ ]:
grid[:3, 1:] = create_expanded_button('I am new too!!!!!', 'warning')

In [ ]:
from ipywidgets import widgets
accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
accordion

In [ ]:
import imagej
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()

In [ ]:
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')
ImagePlusClass = autoclass('ij.ImagePlus')

In [ ]:
import numpy as np
#import imagej.server as isv
h = WindowManager.getCurrentImage()
help(h.setImage)
a = np.ndarray([512, 512], dtype=np.uint8)
b = ij.convert().convert(ij.py.to_java(a), ImagePlusClass)
print(b)
c = ij.py.to_java(a)
print(c)

In [2]:
import numpy as np
import imagej
from jnius import autoclass

ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()

from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')
ImagePlusClass = autoclass('ij.ImagePlus')

The JVM is already running.


NameError: name 'ij' is not defined

In [1]:


img1 = np.ndarray([512, 512], dtype=np.float32)
ijui.show(ij.py.to_java(img1))
viewer_ip = WindowManager.getCurrentImage()


The JVM is already running.


NameError: name 'ij' is not defined

In [ ]:
img2 = np.ndarray([512, 512], dtype=np.float32)
viewer_ip.setImage(ij.convert().convert(ij.dataset().create(ij.py.to_java(img2)), ImagePlusClass))

In [6]:
import h5py, numpy as np
fn = '/media/xiao_usb/3D_trial_reg_scan_id_29773-29873_2020-06-01-20-38-09.h5'
with h5py.File(fn, 'r') as f:
    data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:200, 420:550]
    #data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:450, 320:650]
    eng = f['/registration_results/reg_results/eng_list'][:]*1e3

In [7]:
new_fn = '/media/xiao_usb/test_data_small.h5'
with h5py.File(new_fn, 'w') as f:
    f.create_dataset('spec', data=data.astype(np.float32))
    f.create_dataset('eng', data=eng.astype(np.float32))
